In [10]:
import sched, time
import matplotlib.pyplot as plt
import numpy as np
import gsw
import pandas as pd
from scipy.io import netcdf
import seaborn as sns
sns.set(style='ticks',context='talk',font_scale=1.5)
import cmocean.cm as cmo
import glob

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def latest_netcdf(path,file=-1):
    ndir = glob.glob(path+'full_*.cdf')
    return ndir[file].split('/')[-1]

In [7]:
path = './output_2013asiri_08/'
for file in glob.glob(path+'full_*.cdf'):
    plot_single_snapshot_withrim(file.split('/')[-1] ,path)

plotted 13500
plotted 13525
plotted 13550
plotted 13575
plotted 13600
plotted 13625
plotted 13650
plotted 13675
plotted 13700
plotted 13725
plotted 13750
plotted 13775
plotted 13800
plotted 13825
plotted 13850
plotted 13875
plotted 13900
plotted 13925
plotted 13950
plotted 13975
plotted 14000
plotted 14025
plotted 14050
plotted 14075
plotted 14100
plotted 14125
plotted 14150
plotted 14175
plotted 14200
plotted 14225
plotted 14250
plotted 14275
plotted 14300
plotted 14325
plotted 14350
plotted 14375
plotted 14400
plotted 14425
plotted 14450
plotted 14475
plotted 14500
plotted 14525
plotted 14550
plotted 14575
plotted 14600
plotted 14625
plotted 14650
plotted 14675
plotted 14700
plotted 14725
plotted 14750
plotted 14775
plotted 14800
plotted 14825
plotted 14850
plotted 14875
plotted 14900
plotted 14925
plotted 14950
plotted 14975
plotted 15000
plotted 15025
plotted 15050
plotted 15075
plotted 15100
plotted 15125
plotted 15150
plotted 15175
plotted 15200
plotted 15225
plotted 15250
plotte

In [30]:
def plot_inital_condition(cdffile):
    #cdffile = 'full_17400.cdf'
    zgrid = pd.read_csv(path+'zgrid.out', skipinitialspace=True, sep=' ', header=None)[1][1:51].astype('float')
    time = cdffile.split('.')[0].split('_')[1]
    swapaxesint = int( cdffile.split('_')[1].split('.')[0] )
    with netcdf.netcdf_file(path+cdffile,'r') as f:
        # print(f.variables)
        temptop = f.variables['temp'][:][49,:,:].copy()
        tempface = f.variables['temp'][:][:, :, 1].copy()
        saltop = f.variables['s'][:][49,:, :].copy()
        salface = f.variables['s'][:][:, :, 1].copy()
        rhotop = f.variables['rho'][:][49, :, :].copy()-1000
        rhoface = f.variables['rho'][:][:, :, 1].copy()-1000
        vortop = f.variables['vor'][:][49, :, :].copy()/ gsw.f(17)
        vorface = f.variables['vor'][:][:, :, 1].copy() / gsw.f(17)
        x = f.variables['xc'][:].copy()
        y = f.variables['yc'][:] .copy()

        fig, ((ax1, ax3, ax7), (ax2, ax4, ax8)) = plt.subplots(2, 3, figsize=(34, 15))
        levels=[20,21,22,24,26,27]

        v = np.linspace(17, 29, 101)
        ax1.contourf(y, x, temptop.T, v, cmap=cmo.thermal,extend='both')
        CS = ax1.contour(y,x, rhotop.T,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
#         ax1.set(title='Temperature %s' % time, ylabel='x', xticks=[])
        ax1.set(ylabel='x [km]', xticks=[])

        h1 = ax2.contourf(y, zgrid, tempface, v, cmap=cmo.thermal, extend='both')
        CS = ax2.contour(y, zgrid, rhoface,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax2.set(xlabel='y [km]', ylabel='z [m]', ylim=(-200, 0))
        
        a = ax1.get_position()
        cbar_ax = fig.add_axes([a.bounds[0]+a.bounds[2]+0.01, a.bounds[1], 0.01, a.bounds[3]]) # l,b,w,h
        cbar = fig.colorbar(h1, cax=cbar_ax,ticks=np.arange(15,30,2),label='Temperature [$^\circ{}C$]')
        cbar.ax.set_xticklabels(np.arange(16,30,2))

        ax1.annotate('TOP',(0.02,0.02),xycoords='axes fraction',fontsize=20,color='black',fontweight='bold')
        ax2.annotate('FACE',(0.02,0.02),xycoords='axes fraction',fontsize=20,color='white',fontweight='bold')
        
        v = np.linspace(31.55, 35.2, 101)
        ax3.contourf(y, x, saltop.T, v, cmap=cmo.haline,extend='both')
        CS = ax3.contour(y,x, rhotop.T,levels=levels,cmap='Greys',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax3.set(yticks=[], xticks=[])
#         ax3.set(title='Salinity %s' % time, yticks=[], xticks=[])

        h2 = ax4.contourf(y, zgrid, salface, v, cmap=cmo.haline,extend='both')
        CS = ax4.contour(y, zgrid, rhoface,levels=levels,cmap='Greys',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax4.set(xlabel='y [km]', ylim=(-200, 0), yticks=[])

        a = ax3.get_position()
        cbar_ax = fig.add_axes([a.bounds[0]+a.bounds[2]+0.01, a.bounds[1], 0.01, a.bounds[3]]) # l,b,w,h
        cbar = fig.colorbar(h2, cax=cbar_ax,ticks=np.arange(30,36,1),label='Salinity [psu]')
        cbar.ax.set_xticklabels(np.arange(30,36,1))
        
        v = np.linspace(18.8, 27, 101)
        ax7.contourf(y, x, rhotop.T, v, cmap=cmo.dense, extend='both')
        CS = ax7.contour(y,x, rhotop.T,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax7.set(yticks=[], xticks=[])        
#         ax7.set(title='Density %s' % time, yticks=[], xticks=[])

        h4 = ax8.contourf(y, zgrid, rhoface, v, cmap=cmo.dense, extend='both')
        CS = ax8.contour(y, zgrid, rhoface,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax8.set(xlabel='y [km]', ylim=(-200, 0), yticks=[])

        a = ax7.get_position()
        cbar_ax = fig.add_axes([a.bounds[0]+a.bounds[2]+0.01, a.bounds[1], 0.01, a.bounds[3]]) # l,b,w,h
        cbar = fig.colorbar(h4, cax=cbar_ax,ticks=np.arange(19,26,2),label='density $\sigma$ [kg m$^{-3}$]')
        cbar.ax.set_xticklabels(np.arange(-4,5,2))

        plt.subplots_adjust(wspace=0.2, hspace=0.10)

        filename='inital_s_t_%s.pdf' % time
        plt.savefig(path+'/figures'+filename, bbox_inches='tight',dpi=500)
        print('plotted %s' %time)
        plt.show()
    return

In [56]:
def plot_single_snapshot_withrim(cdffile,path):
    zgrid = pd.read_csv(path+'zgrid.out', skipinitialspace=True, sep=' ', header=None)[1][1:51].astype('float')
    time = cdffile.split('.')[0].split('_')[1]
    swapaxesint = int( cdffile.split('_')[1].split('.')[0] )
    with netcdf.netcdf_file(path+cdffile,'r') as f:
        # print(f.variables)
        temptop = f.variables['temp'][:][49,:,:].copy()
        tempface = f.variables['temp'][:][:, :, 1].copy()
        saltop = f.variables['s'][:][49,:, :].copy()
        salface = f.variables['s'][:][:, :, 1].copy()
        rhotop = f.variables['rho'][:][49, :, :].copy()-1000
        rhoface = f.variables['rho'][:][:, :, 1].copy()-1000
        vortop = f.variables['vor'][:][49, :, :].copy()/ gsw.f(17)
        vorface = f.variables['vor'][:][:, :, 1].copy() / gsw.f(17)
        x = f.variables['xc'][:].copy()
        y = f.variables['yc'][:] .copy()

        fig, ((ax1, ax3, ax7,ax5), (ax2, ax4, ax8,ax6)) = plt.subplots(2, 4, figsize=(42, 15))
        levels=[20,21,22,24,26,27]

        v = np.linspace(23.5, 28., 101)
        #ax1.contourf(y, x, temptop.T, v, cmap=cmo.thermal,extend='both')
        ax1.pcolormesh(y,x,temptop.T, cmap=cmo.thermal,shading='gouraud',vmin=23.5,vmax=28.7)
        CS = ax1.contour(y,x, rhotop.T,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax1.set( ylabel='x', xticks=[])
        h1 = ax2.pcolormesh(y, zgrid, tempface,cmap=cmo.thermal,shading='gourad',vmin=23.5,vmax=28.7)#, v, cmap=cmo.thermal, extend='both')
        CS = ax2.contour(y, zgrid, rhoface,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax2.set(xlabel='y', ylabel='z', ylim=(-200, 0))
        
        a = ax1.get_position()
        cbar_ax = fig.add_axes([a.bounds[0]+a.bounds[2]+0.005, 0.15, 0.005, 0.7]) #
        cbar = fig.colorbar(h1, cax=cbar_ax,ticks=np.arange(15,30,2),label='Temperature [$^\circ{}C$]')
        cbar.ax.set_xticklabels(np.arange(15,30,2))

        ax1.annotate('TOP',(0.02,0.02),xycoords='axes fraction',fontsize=20,color='black',fontweight='bold')
        ax2.annotate('FACE',(0.02,0.02),xycoords='axes fraction',fontsize=20,color='white',fontweight='bold')
        
        v = np.linspace(31.55, 35.1, 101)
        ax3.pcolormesh(y, x, saltop.T,cmap=cmo.haline,shading='gouraud',vmin=31.5,vmax=35.1)# v, cmap=cmo.haline,extend='both')
        CS = ax3.contour(y,x, rhotop.T,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax3.set( yticks=[], xticks=[])
        h2 = ax4.pcolormesh(y, zgrid, salface,cmap=cmo.haline,shading='gouraud',vmin=31.5,vmax=35.1) #, v, cmap=cmo.haline,extend='both')
        CS = ax4.contour(y, zgrid, rhoface,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax4.set(xlabel='y', ylim=(-200, 0), yticks=[])

        a = ax3.get_position()
        cbar_ax = fig.add_axes([a.bounds[0]+a.bounds[2]+0.005, 0.15, 0.005, 0.7]) #
        cbar = fig.colorbar(h2, cax=cbar_ax,ticks=np.arange(30,36,1),label='Salinity [psu]')
        cbar.ax.set_xticklabels(np.arange(30,36,1))

        v = np.linspace(-4, 4, 101)
        ax5.pcolormesh(y, x, vortop.T,cmap=cmo.curl,shading='gouraud',vmin=-4,vmax=4)#, v, cmap=cmo.curl, extend='both')
        CS = ax5.contour(y,x, rhotop.T,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax5.set( yticks=[], xticks=[])
        h3 = ax6.pcolormesh(y, zgrid, vorface,cmap=cmo.curl,shading='gouraud',vmin=-4,vmax=4)#, v, cmap=cmo.curl, extend='both')
        CS = ax6.contour(y, zgrid, rhoface,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax6.set(xlabel='y', ylim=(-200, 0), yticks=[])

        a = ax5.get_position()
        cbar_ax = fig.add_axes([a.bounds[0]+a.bounds[2]+0.005, 0.15, 0.005, 0.7]) #
        cbar = fig.colorbar(h3, cax=cbar_ax,ticks=np.arange(-10,11,2),label='Vorticity/f')
        cbar.ax.set_xticklabels(np.arange(-4,5,2))
        
        v = np.linspace(18.8, 24.8, 101)
        ax7.pcolormesh(y, x, rhotop.T,cmap=cmo.dense,shading='gouraud',vmin=18.8,vmax=24.8)#, v, cmap=cmo.dense, extend='both')
        CS = ax7.contour(y,x, rhotop.T,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax7.set( yticks=[], xticks=[])
        h4 = ax8.pcolormesh(y, zgrid, rhoface, cmap=cmo.dense,shading='gouraud',vmin=18.8,vmax=24.8)#, v, cmap=cmo.dense, extend='both')
        CS = ax8.contour(y, zgrid, rhoface,levels=levels,cmap='Greys_r',linewidths=1)
        plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
        ax8.set(xlabel='y', ylim=(-200, 0), yticks=[])

        a = ax7.get_position()
        cbar_ax = fig.add_axes([a.bounds[0]+a.bounds[2]+0.005, 0.15, 0.005, 0.7]) #
        cbar = fig.colorbar(h4, cax=cbar_ax,ticks=np.arange(18,26,2),label='Density [kg/m$^3$]')
        cbar.ax.set_xticklabels(np.arange(-4,5,2))

#         plt.subplots_adjust(wspace=0.25, hspace=0.10)
        
        filename='movie_08_%s.png' % time
        plt.savefig('./movie_08/'+filename, bbox_inches='tight')
        print('plotted %s' %time)
        plt.show()
    return

In [ ]:
path = './output_2013asiri_09/'
plot_single_snapshot_withrim(latest_netcdf(path,file=25),path)